In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

## Constantes

In [2]:
# mismas constantes que en notebook 1_...
ANIO = 2018
VENTANA = 5

# Carga Mcp, locations and products

In [3]:
with open(f'serializables/Mcp_{ANIO}_{VENTANA}.pkl','rb') as f:
    Mcp = pickle.load(f)

In [4]:
Mcp.sum(), Mcp.shape

(76902, (122, 4864))

In [5]:
with open(f'serializables/locations_{ANIO}_{VENTANA}.pkl','rb') as f:
    locations = pickle.load(f)

In [6]:
with open(f'serializables/products_{ANIO}_{VENTANA}.pkl','rb') as f:
    products = pickle.load(f)

# Calcula diversity, ubiquity, y matrices diagonales inversas

In [7]:
diversity = Mcp.sum(axis = 1)
diversity.shape

(122,)

In [8]:
ubiquity = Mcp.sum(axis = 0)
ubiquity.shape

(4864,)

In [9]:
D_inv = np.diag(1/diversity)
D_inv.shape

(122, 122)

In [10]:
U_inv = np.diag(1/ubiquity)
U_inv.shape

(4864, 4864)

## Calcula Mmonio c, c'

In [11]:
temp_Mcp_1 = Mcp.copy().astype(float)

In [12]:
for i in range(len(products)):
    temp_Mcp_1[:, i] = temp_Mcp_1[:, i]/ubiquity[i]

In [13]:
temp_Mcp_1.dtype, temp_Mcp_1.shape

(dtype('float64'), (122, 4864))

In [14]:
Sccprima = np.matmul(temp_Mcp_1, Mcp.transpose())
Sccprima.shape, Sccprima.dtype

((122, 122), dtype('float64'))

In [15]:
Mmonio_c = np.matmul(D_inv, Sccprima)
Mmonio_c.shape, Mmonio_c.dtype

((122, 122), dtype('float64'))

In [16]:
with open(f'serializables/Mmonio_c_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(Mmonio_c, f)

## Autovalores y autovectores c,c' y ECI

In [17]:
def calc_complexity(mmonio):
    autovalores, autovectores = np.linalg.eig(mmonio)
    second_idx = np.where(autovalores == -np.sort(-autovalores)[1])[0][0]
    
    # el segundo autovector representa la varianza de complejidades, y por lo tanto es el ECI
    complexity = autovectores[:, second_idx].real
    print('Todos los avec imaginarios son cero:', (np.isclose(autovectores[:, second_idx].imag, 0.0)).all())
    print('Mean and std of second autovector:', complexity.mean(), '+/-', complexity.std())
    
    return (complexity - complexity.mean())/complexity.std()
    

In [18]:
eci_norm = calc_complexity(Mmonio_c)

Todos los avec imaginarios son cero: True
Mean and std of second autovector: 0.042074388938290117 +/- 0.08016524874872466


In [19]:
print('Mean and std of eci:', eci_norm.mean(), '+/-', eci_norm.std())

Mean and std of eci: 4.4590924759534976e-17 +/- 0.9999999999999999


In [22]:
df_eci = pd.DataFrame({'Country code': locations, 'eci': eci_norm})
df_eci.sort_values('eci', ascending=False, inplace=True)
df_eci

,Country code,eci
53,JPN,2.414576
14,CHE,2.325609
23,DEU,2.153279
99,SGP,1.916753
114,USA,1.805761
...,...,...
7,BGD,-1.555508
89,PNG,-1.563585
97,SDN,-1.575354
0,AGO,-1.634121


#### Chequear que los ECI no esten al reves. Si estan al reves hay que descomentar la proxima celda para cambiarles el signo. Esto se debe a que el calculo de autovectores tiene su dirección indeterminada, y se debe determinar a mano.

In [21]:
eci_norm = -eci_norm

In [23]:
with open(f'serializables/eci_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(eci_norm, f)

## Calcula Mmonio p, p'

In [24]:
temp_Mcp_2 = Mcp.copy().astype(float)

In [25]:
for i in range(len(locations)):
    temp_Mcp_2[i, :] = temp_Mcp_2[i, :]/diversity[i]

In [26]:
temp_Mcp_2.dtype, temp_Mcp_2.shape

(dtype('float64'), (122, 4864))

In [27]:
Sppprima = np.matmul(Mcp.transpose(), temp_Mcp_2)
Sppprima.shape, Sppprima.dtype

((4864, 4864), dtype('float64'))

In [28]:
Mmonio_p = np.matmul(U_inv, Sppprima)
Mmonio_p.shape, Mmonio_p.dtype

((4864, 4864), dtype('float64'))

In [29]:
with open(f'serializables/Mmonio_p_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(Mmonio_p, f)

## Autovalores y autovectores p,p' y PCI

In [30]:
pci_norm = calc_complexity(Mmonio_p)

Todos los avec imaginarios son cero: True
Mean and std of second autovector: 0.0031990837312377536 +/- 0.013977051496781709


In [31]:
print('Mean and std of pci:', pci_norm.mean(), '+/-', pci_norm.std())

Mean and std of pci: -1.1686558153949016e-17 +/- 1.0


In [32]:
df_pci = pd.DataFrame({'Product code': products, 'pci': pci_norm})
df_pci.sort_values('pci', ascending=False, inplace=True)
df_pci

,Product code,pci
3452,780500,3.452359
3966,846140,3.224919
1485,370291,2.967474
3346,740323,2.967474
178,050300,2.943348
...,...,...
2200,530310,-2.760333
3503,811240,-2.763685
1395,330122,-2.763685
397,120740,-2.787129


In [33]:
algunos_pci_positivos = ['847989', '902790', '902780', '847990', '902730']
df_pci.loc[df_pci['Product code'].isin(algunos_pci_positivos)]

,Product code,pci
4064,847989,2.241124
4612,902790,2.152727
4611,902780,2.104298
4065,847990,2.013471
4608,902730,1.981902


In [34]:
algunos_pci_negativos = ['560721', '460120', '530599', '630510', '530310']
df_pci.loc[df_pci['Product code'].isin(algunos_pci_negativos)]

,Product code,pci
2419,560721,-1.435431
1875,460120,-1.983101
2208,530599,-2.083508
2799,630510,-2.342862
2200,530310,-2.760333


#### Chequear que los PCI no esten al reves. Si estan al reves hay que descomentar la proxima celda para cambiarles el signo, por las mismas razones que en el caso del ECI.

In [35]:
#pci_norm = -pci_norm

In [36]:
with open(f'serializables/pci_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(pci_norm, f)

# Proximity y Density, con productos (como en el paper)

In [37]:
p_RCAmay1 = np.zeros((len(products),))

for prod in range(len(products)):
    p_RCAmay1[prod] = Mcp[:, prod].sum()
    
p_RCAmay1

array([13., 19., 24., ..., 23., 17., 26.])

In [38]:
almost_proximity = np.zeros((len(products), len(products)))

for p1 in range(len(products)-1):
    if p1 % 487 == 0:
        print('.', end=' ')
        
    # como es una matriz simetrica, calcula unicamente la mitad de los valores:
    for p2 in range(p1+1, len(products)):
        almost_proximity[p1, p2] = np.logical_and(Mcp[:, p1], Mcp[:, p2]).sum()/max(p_RCAmay1[p1], p_RCAmay1[p2])

. . . . . . . . . . 

In [39]:
# chequeo que no haya valores incorrectamente imputados
for i in range(len(products)):
    if not np.isclose(almost_proximity[i, :i].sum(), 0.0):
        print('something wrong at row', i)

In [40]:
proximity = almost_proximity + almost_proximity.transpose() + np.diag(np.ones(len(products)))

In [41]:
density_cp = np.matmul(Mcp, proximity)

for i in range(len(products)):
    density_cp[:, i] = density_cp[:, i]/proximity[:, i].sum()

In [42]:
with open(f'serializables/proximity_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(proximity, f)

In [43]:
with open(f'serializables/density_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(density_cp, f)

## Proximity y density de otra forma (con paises, como propone oec.world)

In [44]:
p_RCAmay1_c = np.zeros((len(locations),))

for loc in range(len(locations)):
    p_RCAmay1_c[loc] = Mcp[loc, :].sum()
    
p_RCAmay1_c

array([  38.,  474.,  377.,  448.,  391., 1410.,   77.,  397., 1158.,
        682.,  745.,  159.,  572.,  801.,  745.,  382., 2290.,  229.,
        174.,   74.,  366.,  392., 1238., 1806., 1172.,  516.,   45.,
        209.,  954., 1813.,  946.,  395.,  789., 1660.,   47., 1298.,
        373.,  206.,  105.,  810.,  646., 1220.,  410., 1030.,  879.,
        955., 1526.,  417.,  393.,  724., 2051.,  258.,  672., 1297.,
        266.,  586.,  487.,  443.,  895.,   61.,  306.,  970.,   82.,
         36.,  676., 1165., 1020.,  638.,  559.,  447.,  627.,  473.,
        134.,  113.,  223.,  100.,  614.,  262.,  811.,  100.,  277.,
       1407.,  407.,  574.,  175.,  880.,  907.,  423.,  710.,   77.,
       1403., 1464.,  175.,   67.,  982.,  421.,  245.,  136.,  411.,
        628.,  582.,  816., 1075., 1104., 1138.,  158.,   75.,  153.,
        830., 1535.,  550.,  488.,  683.,  296., 1508.,  292.,   59.,
        887.,  199.,  909.,  231.,  230.])

In [45]:
almost_proximity_c = np.zeros((len(locations), len(locations)))

for l1 in range(len(locations)-1):
    if l1 % 13 == 0:
        print('.', end=' ')
        
    for l2 in range(l1+1, len(locations)):
        almost_proximity_c[l1, l2] = np.logical_and(Mcp[l1, :], Mcp[l2, :]).sum()/max(p_RCAmay1_c[l1], p_RCAmay1_c[l2])

. . . . . . . . . . 

In [46]:
# chequeo
for i in range(len(locations)):
    if not np.isclose(almost_proximity_c[i, :i].sum(), 0.0):
        print('something wrong at row', i)

In [47]:
proximity_c = almost_proximity_c + almost_proximity_c.transpose() + np.diag(np.ones(len(locations)))

In [48]:
density_cp_c = np.matmul(proximity_c, Mcp)

for i in range(len(locations)):
    density_cp_c[i, :] = density_cp_c[i, :]/proximity_c[:, i].sum()

In [49]:
with open(f'serializables/proximity_c_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(proximity_c, f)

In [50]:
with open(f'serializables/density_with_c_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(density_cp_c, f)